In [1]:
%%capture --no-stderr
%pip install -U langgraph langchain langchain-openai

In [2]:
import os
import sys


os.chdir("/teamspace/studios/this_studio/nurse-helper-chatbot")


sys.path.append(os.getcwd())

In [3]:
from llm import llm
from graph import graph
from langgraph.prebuilt import create_react_agent
from langgraph.prebuilt.chat_agent_executor import AgentState
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from utils import get_session_id
from tools.hospital import hospital_cypher_qa
from tools.patient import patient_cypher_qa
from tools.physician import physician_cypher_qa
from langchain_core.tools import tool

@tool
def explore_hospital(question: str) -> str:
    """Provide information about patient questions using Cypher and context from GraphCypherQAChain"""
    return hospital_cypher_qa.invoke(question)

@tool
def explore_patient(question: str) -> str:
    """Provide information about patient questions using Cypher and context from GraphCypherQAChain."""
    return patient_cypher_qa.invoke(question)


tools = [explore_hospital, explore_patient]

 

In [134]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define the agent prompt using ChatPromptTemplate
agent_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
                        You are a nursing expert providing information about nursing care.
                        Be as helpful as possible and return as much information as possible.
                        Only answer questions related to checkups, hospitals, or patients.

                        You must rely solely on the information provided in the context and the tools available to you. 
                        Do not use any pre-trained knowledge.

                        You **must** use the tools provided for every relevant question. 
                        Do not decline to use a tool, as it is essential for generating accurate responses.

                        When generating responses:
                        Always follow the required format strictly.
                        Use the actual Cypher examples from the `GraphCypherQAChain` to construct accurate queries.
                        Validate the information retrieved from the tools before providing a final answer.
                        If the information is insufficient, indicate that further clarification is needed.

                        Use the following format to provide a response:
                        Final Answer: [your response here]
        """),
        ("user", "{input}"),  # Placeholder for new input
        ("placeholder", "{messages}"),  # Use MessagesPlaceholder for chat history
    ]
)

In [4]:
from langchain_core.messages import SystemMessage

system_message = SystemMessage(content="""
You are a nursing expert providing information about nursing care.
Be as helpful as possible and return as much information as possible.
Only answer questions related to checkups, hospitals, or patients.

You must rely solely on the information provided in the context and the tools available to you. 
Do not use any pre-trained knowledge.

You **must** use the tools provided for every relevant question. 
Do not decline to use a tool, as it is essential for generating accurate responses.

When generating responses:
Always follow the required format strictly.
Use the actual Cypher examples from the `GraphCypherQAChain` to construct accurate queries.
Validate the information retrieved from the tools before providing a final answer.
If the information is insufficient, indicate that further clarification is needed.

Use the following format to provide a response:
```
[**Thought**]
[**Tool_used**:e.g. explore_patient]
[**Response**]
```

Begin!

Previous conversation history:
{{chat_history}}

New input: {{input}}
""")

In [118]:
def _modify_state_messages(state: AgentState):
    # Extract chat history and new input from the state
    chat_history = state["messages"]  # Assuming this is a list of messages
    new_input = state.get("input", "")  # Get the new input, default to empty string if not present

    # Invoke the prompt with the extracted information
    return agent_prompt.invoke({
        "input": new_input,            # Pass the new input
        "messages": chat_history,      # Pass the chat history
        "tools": state.get("tools", ""),  # Pass the tools if available
        "tool_names": state.get("tool_names", ""),  # Pass the tool names if available
    }).to_messages()
    



In [5]:
langgraph_agent_executor = create_react_agent(llm, tools, state_modifier=system_message)

In [7]:
# List the physicians specializing in Pulmonology at Penang General Hospital.
# do Lam Wah Ee Hospital exist in the database? May I know its contact number
query = "can u find a patient information and its checkup with which physician attends the checkup for id 123?"

In [46]:
for step in langgraph_agent_executor.stream(
    {"messages": [("human", query)]}, stream_mode="updates"
):
    print(step)

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_gKZoBdOUxFV0PIVRoEYdvhcg', 'function': {'arguments': '{"question": "What is the information for patient with ID 123?"}', 'name': 'explore_patient'}, 'type': 'function'}, {'id': 'call_Gi039IUuIlcLUi9wyQnj7LWi', 'function': {'arguments': '{"question": "Which physician attended the checkup for patient ID 123?"}', 'name': 'explore_hospital'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 306, 'total_tokens': 375, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7a4bb4c4-2203-4aac-a0c9-ee2021bc3bcd-0', tool_calls=[{'name': 'explore_patient',

In [47]:
messages = langgraph_agent_executor.invoke({"messages": [("human", query)]})

messages



> Entering new GraphCypherQAChain chain...


> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (p:patient {Patient_ID: 123})-[:has]->(c:checkup)<-[:attend]-(phy:physician)
RETURN phy.Physician_Name AS PhysicianName, 
       phy.License_Number AS LicenseNumber, 
       phy.Physician_Contact AS PhysicianContactNumber;

Full Context:
[{'PhysicianName': 'Dr. Gan Li Tee', 'LicenseNumber': 55202, 'PhysicianContactNumber': '03-6386655'}, {'PhysicianName': 'Dr. Tan Siew Ling', 'LicenseNumber': 69654, 'PhysicianContactNumber': '03-8970054'}]
Generated Cypher:

MATCH (p:patient {Patient_ID: 123})
RETURN properties(p) AS PatientProperties; 


> Finished chain.
Full Context:
[{'PatientProperties': {'Name': 'Yusof Abdullah', 'Identity_number': '830823-10-5510', 'Age': 41, 'Allergy': 'Peanuts', 'Patient_Contact': '013-8450245', 'Emergency_number': '013-6550048', 'Gender': 'Male', 'Patient_ID': 123, 'Blood_Type': 'B-', 'Chronic_condition': 'Stroke'}}]

> Finished chain.


{'messages': [HumanMessage(content='can u find a patient information and its checkup with which physician attends the checkup for id 123?', additional_kwargs={}, response_metadata={}, id='441c7e1b-3890-4eb5-a6c9-3c497c2681cb'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7npVN0eY6g9KUQiD7lh2bCzN', 'function': {'arguments': '{"question": "Find patient information for ID 123."}', 'name': 'explore_patient'}, 'type': 'function'}, {'id': 'call_rdyseTiMnmdDgzge453OUP8O', 'function': {'arguments': '{"question": "Which physician attends the checkup for patient ID 123?"}', 'name': 'explore_hospital'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 306, 'total_tokens': 372, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mi

In [25]:
RECURSION_LIMIT = 6

try:
    for chunk in langgraph_agent_executor.stream(
        {"messages": [("human", query)]},
        {"recursion_limit": RECURSION_LIMIT},
        stream_mode="values",
    ):
        print(chunk["messages"][-1])
except GraphRecursionError:
    print({"input": query, "output": "Agent stopped due to max iterations."})

content='can u find a patient information and its checkup with which physician attends the checkup for id 123?' additional_kwargs={} response_metadata={} id='3c8d3737-e562-42d7-8895-21b0c2380a63'


content='' additional_kwargs={'tool_calls': [{'id': 'call_TxxYQvzbQePnhGsMETJobUEm', 'function': {'arguments': '{"question": "Find patient information for patient ID 123."}', 'name': 'explore_patient'}, 'type': 'function'}, {'id': 'call_gqNvykjlyc0xqdlfUxCogGCw', 'function': {'arguments': '{"question": "Find checkup details and physician information for patient ID 123."}', 'name': 'explore_hospital'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 373, 'total_tokens': 441, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-014330c2-afae-4463-ab44-dc0485a342c1-0' tool_calls=[{'name': 'explore_patient', 'args': {'question': 'Find pati

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: Length_of_Stay)} {position: line: 4, column: 33, offset: 231} for query: '\nMATCH (p:patient {Patient_ID: 123})-[:has]->(c:checkup)-[:at]->(h:hospital)-[:recruit]->(phy:physician)\nRETURN c.Checkup_ID, c.Admission_Type, c.Checkup_Status, c.Admission_Date, c.Discharge_Date, \n       c.Discharge_Condition, c.Length_of_Stay, c.Diagnosis, c.Treatment, \n       phy.Physician_Name AS PhysicianName, phy.License_Number AS LicenseNumber, \n       phy.Physician_Contact AS PhysicianContactNumber;\n'


Full Context:
[{'c.Checkup_ID': 252, 'c.Admission_Type': 'Urgent', 'c.Checkup_Status': 'Discharged', 'c.Admission_Date': '5/27/2024', 'c.Discharge_Date': '6/1/2024', 'c.Discharge_Condition': None, 'c.Length_of_Stay': None, 'c.Diagnosis': 'Fractured bone', 'c.Treatment': 'Pain management', 'PhysicianName': 'Dr. Norazlina binti Mohd Ali', 'LicenseNumber': 34655, 'PhysicianContactNumber': '03-8221950'}, {'c.Checkup_ID': 252, 'c.Admission_Type': 'Urgent', 'c.Checkup_Status': 'Discharged', 'c.Admission_Date': '5/27/2024', 'c.Discharge_Date': '6/1/2024', 'c.Discharge_Condition': None, 'c.Length_of_Stay': None, 'c.Diagnosis': 'Fractured bone', 'c.Treatment': 'Pain management', 'PhysicianName': 'Dr. Choong Po Keat', 'LicenseNumber': 35958, 'PhysicianContactNumber': '03-4601885'}, {'c.Checkup_ID': 252, 'c.Admission_Type': 'Urgent', 'c.Checkup_Status': 'Discharged', 'c.Admission_Date': '5/27/2024', 'c.Discharge_Date': '6/1/2024', 'c.Discharge_Condition': None, 'c.Length_of_Stay': None, 'c.Diagno

In [48]:
messages = langgraph_agent_executor.invoke({"messages": [("human", query)]})
print(
    {
        "input": query,
        "output": messages["messages"][-1].content,
    }
)





> Entering new GraphCypherQAChain chain...


> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (p:patient {Patient_ID: 123})-[:has]->(c:checkup)-[:at]->(h:hospital)-[:recruit]->(phy:physician)
RETURN c.Checkup_ID AS CheckupID, 
       c.Admission_Date AS AdmissionDate, 
       c.Discharge_Date AS DischargeDate, 
       phy.Physician_Name AS PhysicianName, 
       phy.License_Number AS LicenseNumber;

Generated Cypher:

MATCH (p:patient {Patient_ID: 123})
RETURN properties(p) AS PatientProperties; 

Full Context:
[{'CheckupID': 252, 'AdmissionDate': '5/27/2024', 'DischargeDate': '6/1/2024', 'PhysicianName': 'Dr. Norazlina binti Mohd Ali', 'LicenseNumber': 34655}, {'CheckupID': 252, 'AdmissionDate': '5/27/2024', 'DischargeDate': '6/1/2024', 'PhysicianName': 'Dr. Choong Po Keat', 'LicenseNumber': 35958}, {'CheckupID': 252, 'AdmissionDate': '5/27/2024', 'DischargeDate': '6/1/2024', 'PhysicianName': 'Dr. Gan Li Tee', 'LicenseNumber': 55202}, {'CheckupID': 252, 'Admission

In [6]:

# Define the memory function
def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

# Create the agent executor
agent_executor = RunnableWithMessageHistory(
    langgraph_agent_executor,
    get_memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

from langchain_core.messages import HumanMessage, AIMessage



def generate_response(user_input):
    """
    Create a handler that calls the Conversational agent
    and streams responses to be printed in the console.
    Returns the final output content from the last AIMessage after streaming.
    """
    # Define the messages structure using HumanMessage
    messages = [HumanMessage(content=user_input)]
    
    # Define the configuration with session ID
    config = {"configurable": {"session_id": get_session_id()}} 

    # Initialize a variable to hold the final output content
    final_output_content = ""

    
    # Stream the responses from the agent
    for chunk in agent_executor.stream({"messages": messages}, config=config):
        print(chunk)  # Print each chunk as it is received
        print("----")  # Separator for readability
            
        # Collect messages from the chunk if they exist
        if 'agent' in chunk and 'messages' in chunk['agent']:
            for message in chunk['agent']['messages']:
                if isinstance(message, AIMessage):
                        # Print the content of the AIMessage
                        # print(f"AIMessage: {message.content}")
                        # Update the final output content
                    final_output_content = message.content  # Store the latest AIMessage content

 

    # Return the final output content
    return final_output_content

In [8]:
# user_input = "which hospital offer specialisation in Neurology, Pediatrics and General Surgery?"

response_content = generate_response(query)

# Print the final output
response_content

2025-01-01 06:34:15.119 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-01 06:34:15.120 Session state does not function when running a script without `streamlit run`
2025-01-01 06:34:15.121 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-01 06:34:15.122 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-01 06:34:15.124 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_psYQkyzXtX5dzxDYz69qfIc9', 'function': {'arguments': '{"question": "Find patient information and checkup details for patient ID 123."}', 'name': 'explore_patient'}, 'type': 'function'}, {'id': 'call_djcRFwal6wClzaoBds8VKGlD', 'function': {'arguments': '{"question": "Find which physician attended the checkup for patient ID 123."}', 'name': 'explore_hospital'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 313, 'total_tokens': 385, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9d93df1c-8941-40a9-a0e7-8ad0191c65da-0', tool_calls=[{'name

Error in RootListenersTracer.on_chain_end callback: KeyError('input')


{'agent': {'messages': [AIMessage(content='```\n[**Thought**]\nI have retrieved the patient information and checkup details for patient ID 123, as well as information about the physicians who attended the checkups.\n[**Tool_used**:functions.explore_patient]\n[**Response**]\nThe patient information for ID 123 is as follows:\n\n1. **Checkup ID**: 252\n   - **Patient Name**: Yusof Abdullah\n   - **Diagnosis**: Fractured bone\n   - **Diagnosis Condition**: Moderate\n   - **Treatment**: Pain management\n   - **Medication**: Paracetamol\n   - **Dosage**: 50mg\n   - **Length of Stay**: 5 days\n   - **Admission Date**: 5/27/2024\n   - **Discharge Date**: 6/1/2024\n   - **Room Number**: D346\n\n2. **Checkup ID**: 301\n   - **Patient Name**: Yusof Abdullah\n   - **Diagnosis**: Fractured bone\n   - **Diagnosis Condition**: Moderate\n   - **Treatment**: IV fluids\n   - **Medication**: Citalopram\n   - **Dosage**: 400mg\n   - **Length of Stay**: Not specified\n   - **Admission Date**: 1/6/2024\n   

'```\n[**Thought**]\nI have retrieved the patient information and checkup details for patient ID 123, as well as information about the physicians who attended the checkups.\n[**Tool_used**:functions.explore_patient]\n[**Response**]\nThe patient information for ID 123 is as follows:\n\n1. **Checkup ID**: 252\n   - **Patient Name**: Yusof Abdullah\n   - **Diagnosis**: Fractured bone\n   - **Diagnosis Condition**: Moderate\n   - **Treatment**: Pain management\n   - **Medication**: Paracetamol\n   - **Dosage**: 50mg\n   - **Length of Stay**: 5 days\n   - **Admission Date**: 5/27/2024\n   - **Discharge Date**: 6/1/2024\n   - **Room Number**: D346\n\n2. **Checkup ID**: 301\n   - **Patient Name**: Yusof Abdullah\n   - **Diagnosis**: Fractured bone\n   - **Diagnosis Condition**: Moderate\n   - **Treatment**: IV fluids\n   - **Medication**: Citalopram\n   - **Dosage**: 400mg\n   - **Length of Stay**: Not specified\n   - **Admission Date**: 1/6/2024\n   - **Discharge Date**: Not specified\n   - 